# Jumbo

In [23]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import openpyxl
from openpyxl import load_workbook

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the Jumbo products page
url = "https://www.jumbo.com/producten/?searchType=keyword&searchTerms=jumbo+chocolade+snoep"
driver.get(url)

# Wait for the page to load and accept cookies
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    )
    accept_button.click()
except:
    print("No accept cookies button found.")

# Wait for products to load
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "jum-card"))
)

# Load page source into BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Extract product data
products = []

for product_card in soup.find_all("div", class_="jum-card"):
    # Extract product title
    title_tag = product_card.find("a", class_="title-link")
    title = title_tag.text.strip() if title_tag else "Title not found"

    # Extract promo price
    promo_price_div = product_card.find("div", class_="promo-price")
    promo_price = (
        re.search(r"[\d]+[.,][\d]+", promo_price_div.text.strip()).group()
        if promo_price_div and promo_price_div.text
        else "Promo price not found"
    )

    # Extract price
    price_whole = product_card.find("span", class_="whole")
    price_fractional = product_card.find("span", class_="fractional")
    price = (
        f"{price_whole.text.strip()},{price_fractional.text.strip()}"
        if price_whole and price_fractional
        else "Price not found"
    )

    # Extract weight
    subtitle_div = product_card.find("div", class_="subtitle")
    weight_span = subtitle_div.find("span", class_="text") if subtitle_div else None
    weight = weight_span.text.strip() if weight_span else "Weight not found"

    # Append to products list
    products.append((title, promo_price, price, weight, "Non_Branded", "Jumbo"))

# Write to Excel
timestamp = datetime.now().strftime('%Y-%m-%d')
file_name = "choco.xlsx"

try:
    # Try loading the existing workbook
    workbook = load_workbook(file_name)
    sheet = workbook.active
except FileNotFoundError:
    # If the file does not exist, create a new workbook and sheet
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    # Write the header row
    sheet.append(["Title", "Promo Price", "Price", "Weight", "Brand", "Store", "Timestamp"])

# Write data to the Excel sheet
for product in products:
    sheet.append((*product, timestamp))

# Save the workbook
workbook.save(file_name)

print(f"Extracted {len(products)} products and saved to {file_name}.")
driver.quit()


Extracted 24 products and saved to choco.xlsx.


# Plus

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # Importing openpyxl for Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.plus.nl/producten/snoep-koek-chocolade-chips-noten/chocolade/chocoladesnoepjes?merk=PLUS"
driver.get(url)
time.sleep(5)

# Click the "Weigeren" button to reject cookies on the Plus site
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-cookies-refuse')]")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('a', id=re.compile(".*-produt_item_link")):
    # Extract the product title from the title attribute of the anchor tag
    title = article.get('title', 'Title not found')

    # Extract the price from the price integers and decimals
    price_integer = article.find('div', class_='font-bold product-header-price-integer')
    price_decimals = article.find('div', class_='font-black product-header-price-decimals')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)}{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the previous (old) price from the price-previous div
    previous_price_span = article.find('div', class_='product-header-price-previous')
    if previous_price_span:
        promo_price = previous_price_span.get_text(strip=True)
    else:
        promo_price = 'Promo price not found'

    # Extract the weight from the 'Per 250 g' span
    weight_span = article.find('span', class_='OSFillParent')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Non_Branded", "Plus"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Write product data to Excel
for product in products:
    ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx


#### Plus Rotsjes

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # Importing openpyxl for Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.plus.nl/zoekresultaten?SearchTerm=rotsjes&merk=PLUS"
driver.get(url)
time.sleep(5)

# Click the "Weigeren" button to reject cookies on the Plus site
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-cookies-refuse')]")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('a', id=re.compile(".*-produt_item_link")):
    # Extract the product title from the title attribute of the anchor tag
    title = article.get('title', 'Title not found')

    # Extract the price from the price integers and decimals
    price_integer = article.find('div', class_='font-bold product-header-price-integer')
    price_decimals = article.find('div', class_='font-black product-header-price-decimals')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)}{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the previous (old) price from the price-previous div
    previous_price_span = article.find('div', class_='product-header-price-previous')
    if previous_price_span:
        promo_price = previous_price_span.get_text(strip=True)
    else:
        promo_price = 'Promo price not found'

    # Extract the weight from the 'Per 250 g' span
    weight_span = article.find('span', class_='OSFillParent')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Non_Branded", "Plus"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Write product data to Excel
for product in products:
    ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()

Data has been successfully saved to choco.xlsx


# Dirk

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # Importing openpyxl for Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.dirk.nl/boodschappen/snacks-snoep/chocolade"
driver.get(url)
time.sleep(10)

# Function to automatically scroll down the page
def scroll_to_load_more(driver, wait_time=2, scroll_increment=700, scroll_limit=1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scroll_limit):
        # Scroll down by the specified increment
        driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
        
        # Wait for new products to load
        time.sleep(wait_time)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit the loop if no new content is loaded
        last_height = new_height

scroll_to_load_more(driver, wait_time=2, scroll_increment=700, scroll_limit=1)

# Wait for the label for "Overige chocolade & bonbons" to be clickable and click it
filter_label = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), 'Overige chocolade & bonbons')]")))
filter_label.click()  # Click the label to apply the filter

time.sleep(5)

# Wait for the label for "1 de beste" to be clickable and click it
filter_label = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), '1 de Beste')]")))
filter_label.click()  # Click the label to apply the filter

# Wait for the products to load after filtering
time.sleep(5)

# Wait for a specific element that indicates products are loaded
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//article[@data-product-id]"))
)

# Now scrape the page source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', attrs={'data-product-id': True}):
    # Extract the product title from the 'title' class
    title = article.find('p', class_='title').get_text(strip=True) if article.find('p', class_='title') else 'Title not found'

    # Extract the price from the 'price' class
    price_integer = article.find('span', class_='price-large')
    price_decimals = article.find('span', class_='price-small')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)},{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the promotional price
    promo_price_span = article.find('div', class_='label price-label')
    if promo_price_span:
        # Get the actual promotional price which is nested inside
        promo_price = promo_price_span.find('span', class_='regular-price').find('span').get_text(strip=True)
    else:
        promo_price = 'Promo price not found'

    # Extract the weight from the subtitle span
    weight_span = article.find('span', class_='subtitle')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Non_Branded", "Dirk"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Write product data to Excel
for product in products:
    ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx


#### Dirk Rotsjes


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # For Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.dirk.nl/zoeken/producten/chocolade%20rotsjes"
driver.get(url)
time.sleep(10)

# Now scrape the page source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', attrs={'data-product-id': True}):
    # Extract the product title from the 'title' class
    title = article.find('p', class_='title').get_text(strip=True) if article.find('p', class_='title') else 'Title not found'

    # Extract the price from the 'price' class
    price_integer = article.find('span', class_='price-large')
    price_decimals = article.find('span', class_='price-small')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)},{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the promotional price
    promo_price_span = article.find('div', class_='label price-label')
    if promo_price_span:
        # Get the actual promotional price which is nested inside
        promo_price = promo_price_span.find('span', class_='regular-price').find('span').get_text(strip=True)
    else:
        promo_price = 'Promo price not found'

    # Extract the weight from the subtitle span
    weight_span = article.find('span', class_='subtitle')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Non_Branded", "Dirk"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Write product data to Excel
for product in products:
    ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()

Data has been successfully saved to choco.xlsx


# Vomar

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # For Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.vomar.nl/zoeken?search=g%27woon%20choco"
driver.get(url)
time.sleep(5)

# Click the "Weigeren" button to reject cookies on the Vomar site
try:
    deny_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonDecline")))
    deny_button.click()
except:
    print("No accept cookies button found.")

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

products = []

# Loop through all product articles
for article in soup.find_all('div', class_='col-xs-12 col-md-3 product'):
    # Extract the product title from the 'description' class
    title = article.find('p', class_='description').get_text(strip=True) if article.find('p', class_='description') else 'Title not found'

    # Extract the price from the 'price right' class
    price_integer = article.find('span', class_='large')
    price_decimals = article.find('span', class_='small')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)}{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the promotional price (if applicable, based on previous logic)
    promo_price = 'Promo price not found'  # Placeholder since no promo price was in the provided HTML

    # Weight extraction can be omitted as there is no weight data in the provided HTML
    weight = 'Weight not found'  # Placeholder since no weight was provided

    # Store the extracted information as a tuple
    products.append((title, price, promo_price, weight, "Non_Branded", "Vomar"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Write product data to Excel
for product in products:
    ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx


# Aldi

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.chrome.service import Service
import openpyxl  # For Excel file handling
import os  # For checking if file exists

# Initialize Chrome driver with Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# List of URLs to loop through
urls = [
    "https://www.aldi.nl/zoeken.html?query=rotsjes&searchCategory=Submitted%20Search",
    "https://www.aldi.nl/zoeken.html?query=time4choco&searchCategory=Submitted%20Search&indices%5Bprod_nl_nl_assortment%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_nl_nl_assortment%5D%5Bconfigure%5D%5BhitsPerPage%5D=12&indices%5Bprod_nl_nl_offers%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_nl_nl_offers%5D%5Bconfigure%5D%5BhitsPerPage%5D=12&indices%5Bprod_nl_nl_recipes%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_nl_nl_recipes%5D%5Bconfigure%5D%5BhitsPerPage%5D=12&indices%5Bprod_nl_nl_content%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_nl_nl_content%5D%5Bconfigure%5D%5BhitsPerPage%5D=12&configure%5BclickAnalytics%5D=true"
]

# Define the file name
file_name = "choco.xlsx"

# Check if the Excel file already exists
if os.path.exists(file_name):
    # If the file exists, load it
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active
else:
    # If the file does not exist, create a new workbook and worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Products"
    # Write the headers
    ws.append(["Product Title", "Price", "Promo Price", "Weight", "Branded", "Retailer", "Timestamp"])

# Loop through the URLs and scrape data
for url in urls:
    driver.get(url)

    # Use WebDriverWait to wait for the articles to load
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "mod-article-tile--default")))

    # Get the page source after the JavaScript has rendered the HTML
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # List to hold product data
    products = []

    # Loop through all product articles
    for article in soup.find_all('div', class_='mod-article-tile mod-article-tile--default'):
        # Extract the product title
        title = article.find('span', class_='mod-article-tile__title').get_text(strip=True) if article.find('span', class_='mod-article-tile__title') else 'Title not found'

        # Extract the promotional price if it exists
        promo_price_element = article.find('s', class_='price__previous')
        promo_price = promo_price_element.get_text(strip=True) if promo_price_element else 'Promo price not found'

        # Extract the current price
        current_price_element = article.find('span', class_='price__wrapper')
        current_price = current_price_element.get_text(strip=True) if current_price_element else 'Price not found'

        # Extract the weight
        weight = article.find('span', class_='price__unit').get_text(strip=True) if article.find('span', class_='price__unit') else 'Weight not found'

        # Store the extracted information as a tuple
        products.append((title, current_price, promo_price, weight, "Non_Branded", "Aldi"))

    # Get current timestamp for the data
    timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

    # Write product data to Excel
    for product in products:
        ws.append((*product, timestamp))  # Write product data with timestamp

# Save the workbook to an Excel file
wb.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx


# Albert Heijn

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re  # Importing the regular expression module
from datetime import datetime  # Importing datetime for timestamp
from selenium.webdriver.chrome.service import Service
from openpyxl import Workbook, load_workbook
import os  # For checking if the file exists

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.ah.nl/producten/snoep-chocolade-koek/chocolade/chocoladesnoepjes?merk=AH&kenmerk=prijsfavoriet"
driver.get(url)
time.sleep(5)

# Borrowed the 'very cool :)' accepteer cookies button from TOTO scraper(group assignment)
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "decline-cookies")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', class_='product-card-portrait_root__ZiRpZ'):
    # Extract the price from the aria-label of the sr-only span
    price_span = article.find('span', class_='sr-only')
    if price_span:
        # Use regular expression to extract the numeric price (e.g., 1.99)
        match = re.search(r'[\d]+[.,][\d]+', price_span.get('aria-label'))
        price = match.group() if match else 'Price not found'
    else:
        price = 'Price not found'
        
    # Extract the promo price (if available) from the correct div
    promo_price_span = article.find('div', class_='price-amount_highlight__ekL92')
    if promo_price_span:
        # Use a nested find to get the sr-only span within the promo price div
        promo_price_span_inner = promo_price_span.find('span', class_='sr-only')
        if promo_price_span_inner:
            # Use regular expression to extract the numeric promo price (e.g., 6.53)
            match_promo_price = re.search(r'[\d]+[.,][\d]+', promo_price_span_inner.get('aria-label'))
            promo_price = match_promo_price.group() if match_promo_price else 'Promo price not found'
        else:
            promo_price = 'Promo price not found'
    else:
        promo_price = 'Promo price not found'

    # Extract the product title from the title attribute of the anchor tag
    title_tag = article.find('a', class_='link_root__EqRHd')
    title = title_tag.get('title') if title_tag else 'Title not found'
    
    # Extract the weight from the product-unit-size span
    weight_span = article.find('span', class_='price_unitSize__Hk6E4')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Branded", "AH"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Write the data to an Excel file
file_name = 'choco.xlsx'
sheet_name = 'AH_Choco'

# Check if the file already exists
if os.path.exists(file_name):
    workbook = load_workbook(file_name)
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(sheet_name)
else:
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = sheet_name

# Write the header if the sheet is new
if sheet.max_row == 1:  # Assuming an empty sheet has only one row
    sheet.append(['Title', 'Price', 'Promo Price', 'Weight', 'Category', 'Store', 'Timestamp'])

# Append data to the sheet
for product in products:
    sheet.append([*product, timestamp])

# Save the workbook
workbook.save(file_name)
print(f"Data has been successfully saved to {file_name} in the {sheet_name} tab.")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx in the AH_Choco tab.


## M&M

### AH

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re  # Importing the regular expression module
from datetime import datetime  # Importing datetime for timestamp
from selenium.webdriver.chrome.service import Service
from openpyxl import Workbook, load_workbook
import os  # For checking if the file exists

# Initialize Chrome driver with Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

url = "https://www.ah.nl/producten/snoep-chocolade-koek/chocolade/chocoladesnoepjes?merk=M%26M%27S"
driver.get(url)
time.sleep(5)

# Borrowed the 'very cool :)' accepteer cookies button from TOTO scraper(group assignment)
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "decline-cookies")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', class_='product-card-portrait_root__ZiRpZ'):
    # Extract the price from the aria-label of the sr-only span
    price_span = article.find('span', class_='sr-only')
    if price_span:
        # Use regular expression to extract the numeric price (e.g., 1.99)
        match = re.search(r'[\d]+[.,][\d]+', price_span.get('aria-label'))
        price = match.group() if match else 'Price not found'
    else:
        price = 'Price not found'
        
    # Extract the promo price (if available) from the correct div
    promo_price_span = article.find('div', class_='price-amount_highlight__ekL92')
    if promo_price_span:
        # Use a nested find to get the sr-only span within the promo price div
        promo_price_span_inner = promo_price_span.find('span', class_='sr-only')
        if promo_price_span_inner:
            # Use regular expression to extract the numeric promo price (e.g., 6.53)
            match_promo_price = re.search(r'[\d]+[.,][\d]+', promo_price_span_inner.get('aria-label'))
            promo_price = match_promo_price.group() if match_promo_price else 'Promo price not found'
        else:
            promo_price = 'Promo price not found'
    else:
        promo_price = 'Promo price not found'

    # Extract the product title from the title attribute of the anchor tag
    title_tag = article.find('a', class_='link_root__EqRHd')
    title = title_tag.get('title') if title_tag else 'Title not found'
    
    # Extract the weight from the product-unit-size span
    weight_span = article.find('span', class_='price_unitSize__Hk6E4')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "Branded", "AH"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Write the data to an Excel file
file_name = 'choco.xlsx'
sheet_name = 'AH_Choco'

# Check if the file already exists
if os.path.exists(file_name):
    workbook = load_workbook(file_name)
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(sheet_name)
else:
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = sheet_name

# Write the header if the sheet is new
if sheet.max_row == 1:  # Assuming an empty sheet has only one row
    sheet.append(['Title', 'Price', 'Promo Price', 'Weight', 'Category', 'Store', 'Timestamp'])

# Append data to the sheet
for product in products:
    sheet.append([*product, timestamp])

# Save the workbook
workbook.save(file_name)
print(f"Data has been successfully saved to {file_name} in the {sheet_name} tab.")

# Close the driver
driver.quit()


Data has been successfully saved to choco.xlsx in the AH_Choco tab.


### Jumbo

In [20]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import openpyxl
from openpyxl import load_workbook

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the Jumbo products page
url = "https://www.jumbo.com/producten/menms/?searchType=keyword&searchTerms=m%26m"
driver.get(url)

# Wait for the page to load and accept cookies
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    )
    accept_button.click()
except:
    print("No accept cookies button found.")

# Wait for products to load
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "jum-card"))
)

# Load page source into BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Extract product data
products = []

for product_card in soup.find_all("div", class_="jum-card"):
    # Extract product title
    title_tag = product_card.find("a", class_="title-link")
    title = title_tag.text.strip() if title_tag else "Title not found"

    # Extract promo price
    promo_price_div = product_card.find("div", class_="promo-price")
    promo_price = (
        re.search(r"[\d]+[.,][\d]+", promo_price_div.text.strip()).group()
        if promo_price_div and promo_price_div.text
        else "Promo price not found"
    )

    # Extract price
    price_whole = product_card.find("span", class_="whole")
    price_fractional = product_card.find("span", class_="fractional")
    price = (
        f"{price_whole.text.strip()},{price_fractional.text.strip()}"
        if price_whole and price_fractional
        else "Price not found"
    )

    # Extract weight
    subtitle_div = product_card.find("div", class_="subtitle")
    weight_span = subtitle_div.find("span", class_="text") if subtitle_div else None
    weight = weight_span.text.strip() if weight_span else "Weight not found"

    # Append to products list
    products.append((title, promo_price, price, weight, "branded", "Jumbo"))

# Write to Excel
timestamp = datetime.now().strftime('%Y-%m-%d')
file_name = "choco.xlsx"

try:
    # Try loading the existing workbook
    workbook = load_workbook(file_name)
    sheet = workbook.active
except FileNotFoundError:
    # If the file does not exist, create a new workbook and sheet
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    # Write the header row
    sheet.append(["Title", "Promo Price", "Price", "Weight", "Brand", "Store", "Timestamp"])

# Write data to the Excel sheet
for product in products:
    sheet.append((*product, timestamp))

# Save the workbook
workbook.save(file_name)

print(f"Extracted {len(products)} products and saved to {file_name}.")
driver.quit()


Extracted 17 products and saved to choco.xlsx.


### Plus

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re  # Importing the regular expression module
from datetime import datetime  # Importing datetime for timestamp
from selenium.webdriver.chrome.service import Service
import openpyxl
from openpyxl import load_workbook

# Initialize Chrome driver with Service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.plus.nl/producten/snoep-koek-chocolade-chips-noten/chocolade/chocoladesnoepjes?merk=M%26M%27S"
driver.get(url)

# Click the "Weigeren" button to reject cookies on the Plus site

# Wait for the "Weigeren" button to be clickable
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-cookies-refuse')]")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(3)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('a', id=re.compile(".*-produt_item_link")):
    # Extract the product title from the title attribute of the anchor tag
    title = article.get('title', 'Title not found')

    # Extract the price from the price integers and decimals
    price_integer = article.find('div', class_='font-bold product-header-price-integer')
    price_decimals = article.find('div', class_='font-black product-header-price-decimals')

    if price_integer and price_decimals:
        price = f"{price_integer.get_text(strip=True)}{price_decimals.get_text(strip=True)}"
    else:
        price = 'Price not found'

    # Extract the previous (old) price from the price-previous div
    previous_price_span = article.find('div', class_='product-header-price-previous')
    if previous_price_span:
        # Extract the old price as text
        promo_price = previous_price_span.get_text(strip=True)
    else:
        promo_price = 'Promo price not found'

    # Extract the weight from the 'Per 250 g' span
    weight_span = article.find('span', class_='OSFillParent')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight, "branded", "Plus"))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD HH:MM:SS

# Write the data to an Excel file
file_name = "choco.xlsx"

try:
    # Try loading the existing workbook
    workbook = load_workbook(file_name)
    sheet = workbook.active
except FileNotFoundError:
    # If the file does not exist, create a new workbook and sheet
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    # Write the header row
    sheet.append(["Title", "Price", "Promo Price", "Weight", "Brand", "Store", "Timestamp"])

# Write data to the Excel sheet
for product in products:
    sheet.append((*product, timestamp))

# Save the workbook
workbook.save(file_name)

print(f"Data has been successfully saved to {file_name}")

# Close the driver
driver.quit()

Data has been successfully saved to choco.xlsx
